# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The api, however, provides you with a lot of information that will not be pertinent to your analysis. YOu will pull data from the api and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the api will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

___

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

- Write a function to make a call to the yelp API

In [32]:
import pandas as pd
import requests
import json

In [33]:
with open('../ds-apis-nbz32-main/.secrets/creds.json') as f:
    creds = json.load(f)

In [55]:
api_key = creds['key']
term = 'bike'
location = 'Manhattan NY'
SEARCH_LIMIT = 5
url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'offset': 0 
    }

In [56]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer ' + api_key}
    
    data = requests.get(url, headers = headers, params = url_params)
    return data

In [57]:
results = yelp_call(url_params, api_key)

- Write a function to parse the API response so that you can easily insert the data in to the DB

In [65]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    results = results.json()['businesses']
    parsed_data = []
    for business in results:
        if 'price' not in business:
            biz_info = (business['id'], business['name'], business['is_closed'],
                business['review_count'], business['rating'], 'NaN',
                business['location']['zip_code'])
            parsed_data.append(biz_info)
        else:
            biz_info = (business['id'], business['name'], business['is_closed'],
                business['review_count'], business['rating'], business['price'],
                business['location']['zip_code'])
            parsed_data.append(biz_info)
    return parsed_data

In [59]:
#biker_df = pd.DataFrame(parse_results(results), columns = ('ID', 'Name', 'Is_Closed',
#                                           'Review_Count', 'Rating', 'Price',
#                                           'Zip_Code'))

- Write a function to take your parsed data and add it to the csv file where you will store all of your results. 

In [63]:
#biker_df.to_csv('C:/Users/IM/Documents/Flatiron/ds-east-042621-lectures/Phase_1/phase-1-project-yelp/bikedata.csv', mode='a', index=False, header=False)

In [67]:
parsed_results = parse_results(results)

In [68]:
parsed_results

[('Y2vbaQdhP6ALJrhGYpkFQg',
  'Kickstand Bicycles',
  False,
  161,
  5.0,
  '$$',
  '10017'),
 ('n4YHCDxWp07DicFxOcLzlw', 'Bicycles NYC', False, 125, 4.0, '$$$', '10075'),
 ('XnBflxEeUp0C-MefpSSTMg', 'Liberty Cycles', False, 170, 4.5, '$', '10019'),
 ('HdEwDk5wyjaLIvuhG9Awrw',
  'New Bo Bo Bicycle',
  False,
  116,
  5.0,
  '$',
  '10013'),
 ('wN6qVSygUdRPWblnyqW3mQ',
  'Master Bike Shop',
  False,
  144,
  4.0,
  '$$',
  '10023')]

In [78]:
def df_safe(csv_filepath, parsed_results):
    # your code to open the csv file, concat the current data, and save the data. 
    parsed_results = pd.DataFrame(parsed_results)
    return parsed_results.to_csv(csv_filepath, mode = 'a', index = False, header = False)

In [93]:
csv_filepath = 'C:/Users/IM/Documents/Flatiron/Phase_1_Project/new_york_bike_data.csv'
t_ = df_safe(csv_filepath, parsed_results)

- Write a script that combines the three functions above into a single process.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [81]:
from helper_funcs import *

ModuleNotFoundError: No module named 'helper_funcs'

In [82]:


# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20

NameError: name 'num' is not defined

___

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you ahve a model of the functions you will need to write, and how to pull them together in one script. For this part, you ahve the process below 

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

- Write a function that takes a business id and makes a call to the API for reivews


- Write a function to parse out the relevant information from the reviews

- Write a function to save the parse data into a csv file containing all of the reviews. 

- Combine the functions above into a single script  

___

## Part 4 -  Using python and pandas, write code to answer the questions below. 


- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20